In [87]:
# Import packages
import sys 
sys.path.append('./New functions & Dash GUI')
#import galaxy_functions
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State, no_update, ctx
import dash
import numpy as np
import astropy.units as u
import pandas as pd
import plotly.express as px
import csv
from IPython.display import FileLink
import galaxy_functions
import plotly.graph_objects as go
import importlib

importlib.reload(galaxy_functions)

from galaxy_functions import ElementInput, redshifted_wavelengthFunc, line_flux_at_z, sensScaled, Detected_Galaxies


# print(Detected_Galaxies)
# Incorporate data
line_table_data = np.load('/data/projects/FIRESIDE/data/line_table_cat_v9.npy', allow_pickle=True) #not sure if needed

# Initialize the app
app = Dash()

# App layout
app.layout = [
    html.Div(children='N of Galaxies Detected for Selected Element'),
    html.Hr(),
    html.Div(children=[
    html.Label('Select Element'),
    dcc.Dropdown(options=['[CII] 158', '[OI] 145', '[NII] 122', '[OIII] 88', '[OI] 63', 
    '[NIII] 57', '[OIII] 52', '[SIII] 19', '[NeIII] 16', '[NeII] 13',
   '[OIV] 26', '[SIII] 33', '[SII] 35', '[SIV] 11', 'PAH 7.7',
   'PAH 11.3', 'Bralpha', 'Hmalpha', 'Pfalpa', 'Halpha',
   '[NeV] 14', '[NeV] 24'], value='[CII] 158', id='controls-and-radio-item', multi=True
                )], style={"display":"inline-block", "gridTemplateColumns":"repeat(22, 1fr)", "gridGap":"10px","maxWidth":"100%", "margin":"auto", 'textAlign':"left"}), 
    
    html.Br(),
    html.Div(children=[
    html.Label('Min Wavelength (μm)'), dcc.Input(id='waveMin', type='number', value=25, placeholder='Enter min λ in microns'), 
    html.Label('Max Wavelength (μm)'), dcc.Input(id='waveMax', type='number', value=250, placeholder='Enter max λ in micron'), 
    html.Label(f'Inst. Sensitivity (W/m^2)'), dcc.Input(id='sensitivity', type='number', value=10**-19, placeholder='Enter Instrument Sensitvity in Watts per sqaure meter'), 
    html.Label('Observation Time (hr)'), dcc.Input(id='tobsv', type='number', value=1000, placeholder='Enter Observation Time in Hours'), 
    html.Label(f'FOV (deg^2)'), dcc.Input(id='fov', type='number', value=1.00, placeholder='Enter FOV in sqaure degrees'),
    html.Label(f'Size Observed Area (deg^2)'), dcc.Input(id='area', type='number', value=1.96, placeholder='Enter Size of Observed Area in sqaure degrees'), 
    ], style={"display":"grid", "gridTemplateColumns":"repeat(6, 1fr)", "gridGap":"10px","maxWidth":"1000px", "margin":"auto"}), 
    
    
    

    dcc.Store(id='function_data'), 
    dcc.Store(id='GalaxyCat_data'),
    
#code below needs extra work 
    html.Div(children=[
    html.Br(),
    html.Label('Select X-axis'),
    dcc.Dropdown(['z', 'RA', 'DEC', 'MHALO', 'MSTAR', 'QFLAG', 'SFR', 'MU', 'ISSB', 'UMEAN', 'LIR'], 'LIR', placeholder='Select X-axis', id='dropdown_x'), 
    html.Label('Select Y-axis'),
    dcc.Dropdown(['z', 'RA', 'DEC', 'MHALO', 'MSTAR', 'QFLAG', 'SFR', 'MU', 'ISSB', 'UMEAN', 'LIR'], 'MSTAR', placeholder='Select Y-axis', id='dropdown_y'),
    
    html.Label('Select Graph-type'),
    dcc.Dropdown(['Histogram', 'XY-Graph', '2D-Histogram'], 'Histogram', id='graph_type'),
    dcc.RadioItems(['All Linear', 'All Log', 'X-Log', 'Y-Log'], 'Linear', id='axis_type')
    ]),

    html.Button("Plot Graph", id="graph_button", n_clicks=0),
    
    dcc.Graph(figure={}, id='controls-and-graph'), 
        
    html.Button("CSV Download Filtered Galaxies", id="CSV_Download"), 
    dcc.Download(id='catIndicies_file')
    
    

    
]

# Add controls to build the interaction

#Callback for Function
@callback(
    Output(component_id='function_data', component_property='data'),
    Input(component_id='controls-and-radio-item', component_property='value'), 
    Input(component_id='waveMin', component_property='value'),
    Input(component_id='waveMax', component_property='value'), 
    Input(component_id='sensitivity', component_property='value'),
    Input(component_id='tobsv', component_property='value'),
    Input(component_id='fov', component_property='value'),
    Input(component_id='area', component_property='value'),
    prevent_initial_call='initial_duplicate',
)
def update_function(Element, waveMin, waveMax, sensitivity, tobsv, fov, area):
    if waveMin is None or waveMax is None or sensitivity is None or tobsv is None or area is None or fov is None or Element is None:
        raise dash.exceptions.PreventUpdate
    
    
    waveMin=waveMin*u.micron
    waveMax=waveMax*u.micron
    sensitivity=sensitivity*u.W/u.m**2
    tobsv=tobsv*u.hr
    area=area*u.degree**2
    fov=fov*u.degree**2
    

    panda_dataframe, catIndices = Detected_Galaxies(Element, line_table_data, waveMin, waveMax, sensitivity, tobsv, fov, area)
    return catIndices

In [88]:
@callback(
    Output(component_id='GalaxyCat_data', component_property='data'), 
    Input(component_id='function_data', component_property='data'),
    Input(component_id='dropdown_x', component_property='value'), 
    Input(component_id='dropdown_y', component_property='value'), allow_duplicate=True)


def graph_inputs_from_catalog(data_storage_1, X, Y):

    GalaxyCat_Column_map = {
                                'z':0,
                                'RA':1, 
                                'DEC':2,
                                'MHALO':3,
                                'MSTAR':4,
                                'QFLAG':5, 
                                'SFR':6, 
                                'MU':7,
                                'ISSB':8,
                                'UMEAN':9,
                                'LIR':10,
                        
                                }
    catIndices = data_storage_1
    X_axis = line_table_data[GalaxyCat_Column_map[X], catIndices]
    Y_axis = line_table_data[GalaxyCat_Column_map[Y], catIndices]
    
    return X_axis, Y_axis

    

In [89]:
#Callback graph
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='graph_button', component_property='n_clicks'), 
    State(component_id='function_data', component_property='data'),
    State(component_id='GalaxyCat_data', component_property='data'), 
    State(component_id='graph_type', component_property='value'),
    State(component_id='dropdown_x', component_property='value'), 
    State(component_id='dropdown_y', component_property='value'),
    State(component_id='axis_type', component_property='value'), allow_duplicate=True,
    prevent_initial_call=True
    )

    #still need to connect button to graph
def update_graph(n_clicks, function_data, axis, graph_type, X, Y, axis_type):
    if axis_type == 'All Linear':
        graph_scalex = False
        graph_scaley = False
        x_hist = axis[0]
        y_hist = axis[1]
    elif axis_type == 'All Log':
        graph_scalex = True
        graph_scaley = True
        x_hist = np.log10(axis[0]).tolist()
        y_hist = np.log10(axis[1]).tolist()
    elif axis_type == 'X-Log':
        graph_scalex = True
        graph_scaley = False
        x_hist = np.log10(axis[0]).tolist()
        y_hist = axis[1]
    elif axis_type == 'Y-Log':
        graph_scalex = False
        graph_scaley = True
        x_hist = axis[0]
        y_hist = np.log10(axis[1]).tolist()
    
    if n_clicks is None or n_clicks==0:
        raise dash.exceptions.PreventUpdate
        
    if 'graph_button'==ctx.triggered_id:
    #will change to update with every click/refresh option
        if graph_type=='Histogram':
        
            #low, high = np.percentile(x_val, [0.0, 92.5])
            #x_val_filtered = x_val[(x_val >= low) & (x_val <= high)]
            #nbins=100
            #logbins=np.logspace(np.log10(np.min(x_val)), np.log10(np.max(x_val
            fig = px.histogram(x=x_hist, nbins=100, title=X, log_y=graph_scaley)

            #actually take log of catind, for selection if statement of log
                
            return fig
                
        elif graph_type=='XY-Graph':
            x_value = np.ravel(axis[0])
            y_value = np.ravel(axis[1])
            assert len(x_value) == len(y_value)

            fig = px.scatter(x=x_value, y=y_value, title=f" {Y} vs  {X} ", log_x=graph_scalex, log_y=graph_scaley)
                
            return fig

        elif graph_type=='2D-Histogram':
            fig = px.density_contour([axis[0], axis[1]], x=np.ravel(x_hist), y=np.ravel(y_hist), marginal_x='histogram', marginal_y='histogram')
        

            
            return fig
            

In [90]:
@callback(
    Output(component_id='catIndicies_file', component_property='data'), 
    State(component_id='function_data', component_property='data'), 
    Input(component_id='CSV_Download', component_property='n_clicks'), allow_duplicate=True,
    prevent_initial_call=True)
                 
          
def CSV_Download(function_data, csv_button):
    
    if csv_button is None or csv_button==0:
        raise dash.exceptions.PreventUpdate
        
    if 'CSV_Download'==ctx.triggered_id:
         file=np.savetxt(f'FilteredCatalogIndices{csv_button}.csv', function_data, delimiter=',', fmt='%d')
         
         print('Download complete')
                



# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8193)

Processing element: [CII] 158


logbins=np.logspace(np.log10(np.min(x_val)), np.log10(np.max(x_val)), nbins)
            fig = go.Figure(data=[go.Histogram(x=x_val, xbins=dict(start=logbins[0], end=logbins[-1],
                                                                   size=np.diff(np.log10(logbins))[0]), 
                                               autobinx=False, title=X, log_y=graph_scale)])